# Creating Test Input File for the User

In [1]:
import pandas as pd
df = pd.read_csv('hvac_data_cleaned.csv')

In [2]:
# Save the last 60 rows as a simulated user input file
df.tail(60).to_csv('user_input_test.csv', index=False)
print("Created 'user_input_test.csv' for deployment testing!")

Created 'user_input_test.csv' for deployment testing!


# The Deployment

In [3]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

print("Booting up AI Engine...")
loaded_model = load_model('hvac_lstm_model.h5')

with open('hvac_scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)
print("System Ready! Waiting for data...\n")

Booting up AI Engine...
System Ready! Waiting for data...



In [4]:
def predict_hvac_power(csv_path):
    df = pd.read_csv(csv_path)
    if len(df) < 60:
        return f"Error: The AI needs 60 minutes of history. You only provided {len(df)} minutes."
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    hour = df['timestamp'].dt.hour
    day_of_week = df['timestamp'].dt.dayofweek
    
    df['hour_sin'] = np.sin(2 * np.pi * hour / 24.0)
    df['hour_cos'] = np.cos(2 * np.pi * hour / 24.0)
    df['day_sin'] = np.sin(2 * np.pi * day_of_week / 7.0)
    df['day_cos'] = np.cos(2 * np.pi * day_of_week / 7.0)
    
    features = [
        'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 
        'outside_temp', 'ambient_temp', 'ambient_humidity', 
        'summer_SP_temp', 'winter_SP_temp', 'co2_1', 
        'on_off', 'damper', 'active_power'
    ]
    
    recent_data = df[features].tail(60)
    scaled_input = loaded_scaler.transform(recent_data)
    lstm_input = scaled_input.reshape(1, 60, 13)
    
    scaled_prediction = loaded_model.predict(lstm_input, verbose=0)
    
    dummy_array = np.zeros((1, 13))
    dummy_array[0, -1] = scaled_prediction[0, 0]
    real_power = loaded_scaler.inverse_transform(dummy_array)[0, -1]
    return round(real_power, 2)

In [6]:
# The user simply points the function to their CSV file:

user_file = 'user_input_test.csv'
predicted_power = predict_hvac_power(user_file)

print(f" AI PREDICTION: The HVAC system will demand {predicted_power} kW of Active Power in the next minute.")

 AI PREDICTION: The HVAC system will demand 0.33 kW of Active Power in the next minute.
